In [1]:
import tensorflow as tf

In [10]:
# Convert RLE string to a numpy array
def rle_to_mask(rle_string, width, height):
   
    rows, cols = height, width
    
    if rle_string == -1:
        return np.zeros((height, width))
    else:
        rle_numbers = [int(num_string) for num_string in rle_string.split(' ')]
        rle_pairs = np.array(rle_numbers).reshape(-1,2)
        img = np.zeros(rows*cols, dtype=np.uint8)
        for index, length in rle_pairs:
            index -= 1
            img[index:index+length] = 255
        img = img.reshape(cols,rows)
        img = img.T
        return img

In [11]:
# Returns the pixels that delimit the edges of a region in a mask
def get_mask_origine(mask):
    
    white_pixels = np.array(np.where(mask == 255))
    first_white_pixel = white_pixels[:,0]
    last_white_pixel = white_pixels[:,-1]
    
    return (first_white_pixel[1], first_white_pixel[0]), (last_white_pixel[1], last_white_pixel[0])

In [89]:
def get_single_image_bounding_box(data, imageid, image_path, img_width, img_height, resize, pixels_count):
    
    img_id = imageid.split('_')[0]
    
    img = cv2.imread(image_path + img_id + '.jpg')
    
    # Get RLE encoded masks of an image by its img_id and related labels (Flower, Fish...)
    mask_filtered_byId = data[data['ImageId']==imageid]
    img_mask = mask_filtered_byId['EncodedPixels']
    img_mask = img_mask.values[0]
    img_mask_label = mask_filtered_byId['Label']
    
    # Convert one RLE encoded mask into a binary encoded grid
    one_mask = np.zeros((img_height, img_width))
    one_mask = rle_to_mask(img_mask, img_width, img_height)
    
    # Reduce Mask size
    one_mask = cv2.resize(one_mask, dsize=resize)
    
    one_mask_pixels_count = np.count_nonzero(one_mask == 255)
    
    # Find contours in the binary mask
    contours, _ = cv2.findContours(one_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    tmp_edges = []
    for contour in contours:
        # Get the bounding box of the contour
        x, y, w, h = cv2.boundingRect(contour)
        tmp_edges.append({'left':x, 'top':y, 'width':w, 'height':h})
    
    edges = pd.DataFrame(tmp_edges)
 
    left = edges['left'].min()
    right = edges[['left', 'width']].sum(axis=1).max()
    top = edges['top'].min()
    bottom = edges[['top', 'height']].sum(axis=1).max()
    
    x = (left + right) / 2
    y = (top + bottom) / 2
    w = right - left
    h = bottom - top
    
    bbox = {'X': x, 'Y': y, 'W': w, 'H': h}
    
    resized_img = cv2.imread(image_path + 'small/' + img_id + '.jpg')
    
    return imageid, bbox, one_mask, resized_img, one_mask_pixels_count, w * h


In [3]:
# A SUPPRIMER !?
def _displayMasks(imageid, ax, masks, w, h, cmap, alpha, image_path):
    
    img_id = imageid.split('_')[0]
    
    image_path = image_path
    img = cv2.imread(image_path + img_id + '.jpg')
    
    # Get RLE encoded masks of an image by its imageid and related labels (Flower, Fish...)
    masks_filtered_byId = masks[masks['ImageId']==imageid]
    img_masks = masks_filtered_byId['EncodedPixels'].tolist()
    img_masks_labels = masks_filtered_byId['Label'].tolist()

    # Convert RLE encoded masks into a binary encoded grids
    all_masks = np.zeros((h, w))
    one_mask = np.zeros((h, w))
    mask_origines = []
    for rle_mask in img_masks:
        one_mask = rle_to_mask(rle_mask, w, h)
        mask_origines.append(get_mask_origine(one_mask))
        all_masks += one_mask
      
    # Displays images and related masks
    ax.axis('off')

    for origine, label in zip(mask_origines, img_masks_labels):
        ax.annotate(text=label + " 0", xy=origine[0], xytext=(20, -40), xycoords='data', color='yellow', fontsize=10, fontweight='bold', textcoords='offset pixels', arrowprops=dict(arrowstyle="-|>", color='yellow'))
        ax.annotate(text=label + " 1", xy=origine[1], xytext=(-100, 20), xycoords='data', color='yellow', fontsize=10, fontweight='bold', textcoords='offset pixels', arrowprops=dict(arrowstyle="-|>", color='yellow')) 

    ax.set_title(imageid)
    
    ax.imshow(img)
    ax.imshow(all_masks, cmap=cmap, alpha=alpha)

In [2]:
# Displays one mask on top of its originate image
def displayMask(imageid, ax, masks, w, h, image_path, hide_axis=False, show_mask=False):

    cmap = 'viridis'
    alpha = 0.2

    img_id = imageid.split('_')[0]
    
    image_path = image_path
    img = cv2.imread(image_path + img_id + '.jpg')

    if show_mask:
        # Get RLE encoded masks of an image by its imageid and related labels (Flower, Fish...)
        masks_filtered_byId = masks[masks['ImageId']==imageid]
        img_masks = masks_filtered_byId['EncodedPixels'].tolist()
        img_masks_labels = masks_filtered_byId['Label'].tolist()
    
        # Convert RLE encoded masks into a binary encoded grids
        all_masks = np.zeros((h, w))
        one_mask = np.zeros((h, w))
        mask_origines = []
        for rle_mask in img_masks:
            one_mask = rle_to_mask(rle_mask, w, h)
            mask_origines.append(get_mask_origine(one_mask))
            all_masks += one_mask

    # Displays images and related masks
    if hide_axis:
        ax.axis('off')

    if show_mask:
        # Displays images and related masks
        for origine, label in zip(mask_origines, img_masks_labels):
            ax.annotate(text=label + " 0", xy=origine[0], xytext=(20, -40), xycoords='data', color='yellow', fontsize=10, fontweight='bold', textcoords='offset pixels', arrowprops=dict(arrowstyle="-|>", color='yellow'))
            ax.annotate(text=label + " 1", xy=origine[1], xytext=(-100, 20), xycoords='data', color='yellow', fontsize=10, fontweight='bold', textcoords='offset pixels', arrowprops=dict(arrowstyle="-|>", color='yellow')) 

    ax.set_title(imageid)
    
    ax.imshow(img)

    if show_mask:
        ax.imshow(all_masks, cmap=cmap, alpha=alpha)

In [66]:
def reduceImages(imageid, image_path, size):
    
    image_path = image_path
    img = cv2.imread(image_path + imageid + '.jpg')
    img = cv2.resize(img, dsize=size)
    
    return img

In [102]:
def displayBoundingBox(imageid, ax, x, y, w, h):
   
    img_id = imageid.split('_')[0]
    im = cv2.imread('images/small/' + img_id + '.jpg')

    x1 = x - w/2
    x2 = x + w/2
    y1 = y - h/2
    y2 = y + h/2

    #ax.axis('off')
    ax.set_title(imageid)
    ax.imshow(im)
    ax.plot([x1, x2 ,x2, x1, x1],[y1, y1, y2, y2, y1],'yellow')

In [3]:
def load_image(imageid, filepath, isTrain):
    
    if isTrain:
        img_id = tf.strings.split(imageid, sep="_")[0]
    else:
        img_id = imageid.split('_')[0]
    
    img = tf.io.read_file(filepath + img_id + '.jpg')
    img = tf.image.decode_png(img, channels=3)
    
    return img

In [9]:
import time

def predict_image(img, model, show=False):

    t0 = time.time()
    x, y, w, h, p = model.predict(np.array([img], dtype=np.float32))[0]
    
    if show:
        
        # Dé-standardiser
        x = x * 525
        y = y * 350
        w = w * 525
        h = h * 350

        x1 = x - w/2
        x2 = x + w/2
        y1 = y - h/2
        y2 = y + h/2
        
        fig = plt.figure(figsize=(8, 8))
        plt.imshow(img)
        plt.plot([x1, x2 ,x2, x1, x1],[y1, y1, y2, y2, y1],'yellow')
        plt.show()

        print(x, y, w, h)
        print("Execution time: ",time.time()-t0,"secondes")
        print("Probability: ", p)
    
    return p

In [2]:
def markDuplicate(data, group_field, count_field):
    g = pd.DataFrame(data.groupby([group_field]).agg({count_field:'count'}).rename({count_field:'Count'}, axis=1))
    g.reset_index(drop=False, inplace=True)
    l = list(g[g['Count'] > 1]['FileId'])
    data['Multiple'] = data['FileId'].apply(lambda fieldid: True if fieldid in l else False )
    return data

In [4]:
def generateMaskFileFromRLE(imageid, size):
    rle_string = df[df['ImageId']==imageid]['EncodedPixels'].tolist()[0]
    mask_img = rle_to_mask(rle_string, 2100, 1400)
    mask_img = cv2.resize(mask_img, dsize=size)
    
    
    # Start Experience #
    img_id = imageid.split('_')[0]
    img = cv2.imread('images/square/' + img_id + '.jpg')
    mask_img = np.expand_dims(mask_img, axis=-1)
    mask_img = cv2.bitwise_and(img, img, mask=mask_img)
    # End experience #
    
    #new_path = 'images/small/masks/' + imageid + '_mask.jpg'
    new_path = 'images/square/' + imageid + '_mask.jpg'
    if not(os.path.exists(new_path)):
        cv2.imwrite(new_path, mask_img)
    
    return mask_img

In [2]:
import tensorflow as tf

def loadImages(img_id, mask_id):
    
    #img_filepath = 'images/small/' + img_id + '.jpg'
    #mask_filepath = 'images/small/masks/' + mask_id + '_mask.jpg'
    
    img_filepath = 'images/square/' + img_id + '.jpg'
    mask_filepath = 'images/square/' + mask_id + '_mask.jpg'

    img = tf.io.read_file(img_filepath)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [350, 525])

    mask = tf.io.read_file(mask_filepath)
    mask = tf.io.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, [350, 525], method = 'nearest')
   
    # Normilalize
    img = tf.cast(img, tf.float32)/255
    mask = tf.cast(mask, tf.float32)/255

    return img, mask

2023-11-24 08:46:53.800358: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
import tensorflow as tf

def loadImages_square(img_id, mask_id):
    
    img_filepath = 'images/square/' + img_id + '.jpg'
    mask_filepath = 'images/square/' + mask_id + '_mask.jpg'

    img = tf.io.read_file(img_filepath)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [128, 128])

    mask = tf.io.read_file(mask_filepath)
    mask = tf.io.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, [128, 128], method = 'nearest')
   
    # Normilalize
    img = tf.cast(img, tf.float32)/255
    mask = tf.cast(mask, tf.float32)/255

    return img, mask

In [1]:
# Filter Image pixels with Mask and return statistics
def filterWithMask(imageid, dataset, w, h, image_path, show=False):
    
    img_id = imageid.split('_')[0]
    img = cv2.imread(image_path + img_id + '.jpg')
    
    dataset = dataset[dataset['ImageId']==imageid]
    rle = dataset['EncodedPixels'].values[0]
    label = dataset['Label'].values[0]

    one_mask = rle_to_mask(rle, w, h)

    indices = np.argwhere(one_mask == 255)
    selected_pixels = img[indices[:, 0], indices[:, 1]]

    mean_value = np.mean(selected_pixels)
    std_value = np.std(selected_pixels)

    if show:
        one_mask = np.expand_dims(one_mask, axis=-1)
        new_img = np.where(one_mask == 255, img, 0)
        
        plt.imshow(new_img)
        plt.title(imageid)
        plt.axis('off')
        plt.show()

    return round(mean_value, 2), round(std_value, 2)

In [ ]:
# Count non-significant pixels
def countNonSignificantPixels(imageid, image_path, show=False):
    
    img_id = imageid.split('_')[0]
    img = cv2.imread(image_path + img_id + '.jpg')

    # Images couleur converties en niveaux de gris
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Images en niveaux de gris converties en imahges binaires
    _, binary_img = cv2.threshold(img, 1, 255, cv2.THRESH_BINARY)

    black_pixel_count = img.size - cv2.countNonZero(binary_img)

    if show:
        plt.imshow(img)
        plt.axis('off')
        plt.show()
    
    return black_pixel_count, round(black_pixel_count / img.size, 2)

In [ ]:
def showImages(ImageIds, grid_x, grid_y, df, img_width, img_height, image_path, hide_axis=True, show_mask=False):
    
    fig, axes = plt.subplots(grid_x, grid_y, figsize=(20, 10), layout='constrained')
    for axe, img_id in zip(axes.flat, ImageIds):
        displayMask(img_id, axe, df, img_width, img_height, image_path, hide_axis, show_mask)
    plt.show()